In [1]:
import pandas as pd
from openpyxl import Workbook
from openpyxl.styles import Font, PatternFill, Alignment
import ast
import re
import unicodedata

In [2]:
import pandas as pd
from openpyxl import Workbook
from openpyxl.styles import Font, PatternFill, Alignment
import ast, re

# Input file
input_file = "websites_schools_results.xlsx"
output_file = "sample.xlsx"

# Load data
df = pd.read_excel(input_file)

df["domain"] = df["domain"].fillna("")

# Create workbook
wb = Workbook()
ws = wb.active

row_cursor = 1  # keep track of where we are in the new sheet

# Définir des styles
title_fill = PatternFill(start_color="4F81BD", end_color="4F81BD", fill_type="solid")  # bleu
header_fill = PatternFill(start_color="BDD7EE", end_color="BDD7EE", fill_type="solid")  # bleu clair
label_fill = PatternFill(start_color="F2F2F2", end_color="F2F2F2", fill_type="solid")  # gris clair

title_font = Font(color="FFFFFF", bold=True, size=14)
header_font = Font(color="000000", bold=True)
label_font = Font(color="000000", bold=True)

def normalize_text(text):
    """Supprime les accents et caractères spéciaux d'une chaîne"""
    if not isinstance(text, str):
        return ""
    # Transformer en forme décomposée puis supprimer les diacritiques
    text = unicodedata.normalize("NFD", text)
    text = "".join(c for c in text if unicodedata.category(c) != "Mn")
    # Remplacer espaces/tirets par un seul séparateur
    text = text.replace(" ", "").replace("-", "")
    return text.lower()

def detect_email_pattern(emails_list):
    """Détecte le pattern dominant avec priorité stricte"""
    if not emails_list:
        return ""

    samples = [e.split("@")[0] for e in emails_list if "@" in e]

    # priorité : prenom.nom > p.nom > premiere_lettre_prenomNom
    if any(re.match(r"^[a-z]+\.[a-z]+$", s) for s in samples):
        return "prenom.nom"
    if any(re.match(r"^[a-z]\.[a-z]+$", s) for s in samples):
        return "premiere_lettre_prenom.nom"
    if any(re.match(r"^[a-z]+$", s) for s in samples):
        return "premiere_lettre_prenomNom"

    return ""


def generate_email(name, domain, pattern):
    """Construit un email à partir d'un nom complet, d'un domaine et d'un pattern"""
    if not isinstance(name, str) or not name.strip():
        return None
    
    parts = name.strip().split()
    if len(parts) < 2:
        return None
    
    prenom = normalize_text(parts[0])
    nom = normalize_text(parts[-1])

    if pattern == "prenom.nom":
        local = f"{prenom}.{nom}"
    elif pattern == "premiere_lettre_prenom.nom":
        local = f"{prenom[0]}.{nom}"
    elif pattern == "premiere_lettre_prenomNom":
        local = f"{prenom[0]}{nom}"
    else:
        local = f"{prenom}.{nom}"  # fallback

    return f"{local}@{domain}"


for _, row in df.iterrows():
    name = row["name"]
    website = row["website"]
    domain = row["domain"]
    emails = row["emails"]
    staff_list = row["staff"]
    
    email_list = []

    if isinstance(emails, str):
        emails = emails.strip()
        if emails.lower() not in ("", "set()"):
            try:
                parsed = ast.literal_eval(emails)  # parse string into Python object
                if isinstance(parsed, list):
                    for item in parsed:
                        if isinstance(item, set):
                            for val in item:
                                email_list.append(val.strip("'").strip())
                        elif isinstance(item, str):
                            email_list.append(item.strip("'").strip())
                elif isinstance(parsed, set):
                    for val in parsed:
                        email_list.append(val.strip("'").strip())
            except Exception as e:
                print("Parsing error:", e)
                email_list = []

    # --- Garder uniquement les emails du domaine ---
    if isinstance(domain, str) and domain:
        email_list = [e for e in email_list if e.endswith("@" + domain)]
        
    # --- Collect staff emails ---
    staff_emails = []
    if isinstance(staff_list, str):
        try:
            staff_list = ast.literal_eval(staff_list)
        except:
            staff_list = []

    if isinstance(staff_list, list):
        for staff in staff_list:
            if staff.get("email"):
                staff_emails.append(staff["email"])

    # --- Priorité : utiliser staff_emails si dispo, sinon email_list ---
    emails_for_pattern = staff_emails if staff_emails else email_list
    pattern = detect_email_pattern(emails_for_pattern)
    
    # --- First row: Name spanning 4 columns ---
    ws.merge_cells(start_row=row_cursor, start_column=1, end_row=row_cursor, end_column=4)
    cell = ws.cell(row=row_cursor, column=1, value=name)
    cell.fill = title_fill
    cell.font = title_font
    cell.alignment = Alignment(horizontal="center")
    row_cursor += 1

    # --- Website row ---
    ws.cell(row=row_cursor, column=1, value="Site web").fill = label_fill
    ws.cell(row=row_cursor, column=1).font = label_font
    ws.cell(row=row_cursor, column=2, value=website)
    row_cursor += 1

    # --- Domain row ---
    ws.cell(row=row_cursor, column=1, value="Email domaine").fill = label_fill
    ws.cell(row=row_cursor, column=1).font = label_font
    ws.cell(row=row_cursor, column=2, value=domain)
    row_cursor += 1
    
    print(domain, name)


    # --- Emails row ---
    ws.cell(row=row_cursor, column=1, value="Emails scrapés").fill = label_fill
    ws.cell(row=row_cursor, column=1).font = label_font
    if isinstance(emails, str) and emails.strip().lower() not in ("set()", ""):
        ws.cell(row=row_cursor, column=2, value=emails)
    else:
        ws.cell(row=row_cursor, column=2, value="")
    row_cursor += 1

    # --- Email shape suggested row ---
    ws.cell(row=row_cursor, column=1, value="Forme de l'email suggéré").fill = label_fill
    ws.cell(row=row_cursor, column=1).font = label_font
    ws.cell(row=row_cursor, column=2, value=pattern)
    row_cursor += 1

    # --- Staff table header ---
    headers = ["Nom complet", "Rôle", "Email trouvé", "Email suggéré"]
    for col, h in enumerate(headers, start=1):
        c = ws.cell(row=row_cursor, column=col, value=h)
        c.fill = header_fill
        c.font = header_font
        c.alignment = Alignment(horizontal="center")
    row_cursor += 1

    # --- Staff rows ---
    if isinstance(staff_list, str):
        try:
            staff_list = ast.literal_eval(staff_list)
        except:
            staff_list = []
    
    if isinstance(staff_list, list):
        for staff in staff_list:
            ws.cell(row=row_cursor, column=1, value=staff.get("name"))
            ws.cell(row=row_cursor, column=2, value=staff.get("function"))
            ws.cell(row=row_cursor, column=3, value=staff.get("email"))

            suggested = None
            if not staff.get("email") and domain:  # only if email is missing/None/empty
                suggested = generate_email(staff.get("name", ""), domain, pattern)

            ws.cell(row=row_cursor, column=4, value=suggested if suggested else "")
            row_cursor += 1

    # --- Leave two empty rows between blocks ---
    row_cursor += 2

# Ajuster largeur colonnes
for col in range(1, 5):
    ws.column_dimensions[chr(64 + col)].width = 30

# Save
wb.save(output_file)
print(f"✅ File created with email patterns: {output_file}")


purpan.fr École d’Ingénieurs de PURPAN
esc-cybersecurity.com European School of Cybersecurity
dsti.institute DSTI School of Engineering - French Riviera
dsti.institute DSTI School of Engineering - Paris
difcam.com CFA DIFCAM Ile-de-France
purple-campus.com Purple Campus - Alès
iffp.pro IFFP - Institut Français de Formation Professionnelle
purple-campus.com Purple Campus - Nîmes
carcassonne-agglo.fr Purple Campus - Carcassonne
purple-campus.com Purple Campus - Narbonne
isal-paris.fr ISAL PARIS
ief2i.fr Digital School of Paris
fms-school.com SMBS - L'école de commerce de la santé - Paris
 emlyon business school - Campus de Lyon
digital-college.fr Digital College - Lyon
digital-college.fr Digital College - Paris La Défense
 301
efrei.fr Efrei - Ecole d'ingénieur - Informatique et technologies du numérique - Paris
clermont-sb.fr Clermont School of Business
afip-formations.com AFIP Formations
supexpertise.fr Sup'Expertise
fms-school.com FMS - L’école de commerce du Food Business - Paris
mbs